In [228]:
import os, csv

In [271]:
def arrayParser(arr):
    # CSV usage: cast string list to python list
    smooth_stage_1 = arr.replace('[', '').replace(']', '').split(',')
    smooth_stage_2 = map(lambda unit: float(unit), smooth_stage_1)
    return smooth_stage_2

In [272]:
class LVQNeuron:
    def __init__(self, name):
        self.name = name
        self.weights = []
    
    def setWeights(self, weights):
        weights = weights.replace('[', '').replace(']', '').split(',')
        
        for weight in weights:
            self.weights.append(float(weight))

    def __len__(self):
        return len(self.weights)

In [273]:
class LVQNet:
    def __init__(self, inCount, outCount):
        self.inputs   = inCount
        self.outputs  = outCount
        self.alpha    = 0.1
        self.csvCount = 0  # Limit the csvs input to number of output neurons
        self.neurons  = {} # Numbered index map (to outputs)
              
        for n in range(outCount):
            curr_neuron = LVQNeuron(n)
            self.neurons[n] = curr_neuron
            
    def __len__(self):
        return len(self.neurons)
        
    # STEP 0
    def enterCSV(self, filepath): 
        if self.csvCount >= self.outputs:
            print "Reached limit of neurons" # (TODO) Throw error
            return
            
        with open(filepath, 'r') as f:
            read = csv.reader(f, delimiter=',')
            row = read.next()
            curr_neuron = self.neurons[self.csvCount]
            curr_neuron.setWeights(row[1])
            self.csvCount += 1
            
            print "Successfully added neuron from CSV", filepath
            return

    # STEP 3.1
    def edist(self, inputs, weights):
        # Euclidean Distance helper function 
        euclideanDistance = 0
        
        if len(inputs) != len(weights):
            print len(inputs), "different length than", len(weights) # Error
            return
        
        for i in range(len(inputs)):
            nth = inputs[i] - weights[i]
            nth = nth ** 2
            euclideanDistance += nth
             
        return euclideanDistance ** (0.5)
    
    # STEP 3.2 -- (TODO) Test
    def minDist(self, inputVector):
        scores = [] # Euclidean Distances
        
        for neuron in self.neurons:
            wunit = self.neurons[neuron].weights
            scores.append(self.edist(inputVector, wunit))
        
        minNeuronIndex = scores.index(min(scores))
        return minNeuronIndex
    
    # STEP 4 -- In progress ...  
    def calibrate(self, neuronNo, guessNo):
        if neuronNo == guessNo: # replace this with LVQData Map lookup
            print "Right Guess!"
            for weight in self.neurons[neuronNo]:
                print weight,                    # put the math here
        else:
            print "Wrong Guess :("
            for weight in self.neurons[neuronNo]:
                print weight,                    # put other math here

In [286]:
class LVQData:
    def __init__(self):
        self.data          = [] # list of tuples 
        self.instrumentMap = {} # map integers with instruments (labels)
        self.instrumentNum = 0  # current integer instrument (to neuron)
    
    def loadCSV(self, filepath, label):
        # Will skip first line of each CSV since LVQ initializes using the first lines
        with open(filepath, 'r') as f:
            read = csv.reader(f, delimiter=',')
            read.next()
            
            for row in read:
                # Tuple with STFT bins and then the label
                data_struct = (arrayParser(row[1]), label)
                self.data.append(data_struct)
        
            self.instrumentMap[self.instrumentNum] = label
            self.instrumentNum += 1
            
        return self.data
    
    def getVector(self, index):
        return self.data[index][0]
    
    def getVectorLabel(self, index):
        return self.data[index][1]
    
    def lookupInstrument(self, index):
        return self.instrumentMap[index]

In [287]:
if __name__ == '__main__':
    ### Driver: Outline of the API / Algorithm in use 
    
    # Create Network with in and out neuron parameters
    koho = LVQNet(1025, 2)
    
    # Enter data (1-1 CSV to Output Neurons) Initializes the neurons with first onset
    koho.enterCSV('./data/snareFrames.csv')
    koho.enterCSV('./data/kickDrumFrames.csv')
    
    # Instantiate LVQ Training Data Structure and load rest of CSVs with labels
    dataset = LVQData()
    dataset.loadCSV('./data/snareFrames.csv', 'snare')
    dataset.loadCSV('./data/kickDrumFrames.csv', 'kick-drum')
    
    # (TODO) Put the koho minDists in loop / logic
    guess = koho.minDist(dataset.getVector(2)) # using a specific sample frame (snare)
    print dataset.lookupInstrument(guess),
    print dataset.getVectorLabel(2)            # and the supervised answer
    
    guess2 = koho.minDist(dataset.getVector(8)) # using another sample frame (kick)
    print dataset.lookupInstrument(guess2),
    print dataset.getVectorLabel(8)             # and the supervised answer

SyntaxError: invalid syntax (<ipython-input-287-f60cd44c7768>, line 18)